# ACO Hyperparameter Tuning with Optuna

This notebook optimizes ACO parameters for graph coloring using Optuna's Tree-structured Parzen Estimator (TPE).

**Features:**
- **Comprehensive Warm-up Phase**: Exhausts all greedy possibilities (s-start + strategies) before ACO refinement
- **2-Group Post-Warmup Strategy**: Configurable exploitation/exploration balance via exploitation_ratio
- **Enhanced Parameter Space**: Includes heuristic weights, pheromone bounds, strategy configuration, and warm-up control

## 1. Environment Setup

In [1]:
import sys
import os

# Check if running in Google Colab environment
IS_COLAB = 'google.colab' in sys.modules

print(f"Running in Google Colab: {IS_COLAB}")

if IS_COLAB:
    print("Colab environment detected. Will mount Google Drive.")
    # Mount Google Drive if running in Colab
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully at /content/drive")
else:
    print("Local environment detected. Using local paths.")

Running in Google Colab: False
Local environment detected. Using local paths.


In [2]:
# Configure paths for data, studies, results, and figures based on the execution environment.
from pathlib import Path

# Configure base paths based on environment
if IS_COLAB:
    # Update this path to match your Google Drive structure
    BASE_PATH = Path('/content/drive/MyDrive/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'
    # Add code path to system path for imports
    sys.path.insert(0, str(CODE_PATH))
else:
    # Local environment paths
    BASE_PATH = Path('/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'

# Define data root path (contains tiny_dataset and main_dataset)
DATA_ROOT = BASE_PATH / 'data'

# Verify paths exist
if not BASE_PATH.exists():
    raise FileNotFoundError(f"Base path does not exist: {BASE_PATH}")
if not DATA_ROOT.exists():
    raise FileNotFoundError(f"Data root does not exist: {DATA_ROOT}")

print(f"Base Path: {BASE_PATH}")
print(f"Code Path: {CODE_PATH}")
print(f"Data Root: {DATA_ROOT}")
print(f"\nPath verification: OK")

Base Path: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3
Code Path: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code
Data Root: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data

Path verification: OK


In [3]:
# Install required packages if running in Colab
if IS_COLAB:
    print("Installing required packages...")
    !pip install -q networkx==3.2.1 matplotlib==3.8.2 pandas==2.1.4 numpy==1.26.2 optuna==3.5.0 scikit-learn==1.4.0 scipy==1.11.4
    print("Packages installed successfully!")

In [4]:
# Import Required Libraries
from pathlib import Path
import multiprocessing
from datetime import datetime
from IPython.display import Image, display, Markdown, HTML

# Import project modules
from dataloader import GraphDataLoader
from optuna_tuner import OptunaACOTuner
from aco_gpc import ACOGraphColoring
from objective_function import aco_objective_function

print("All libraries imported successfully!")

/Users/mahdy/projects/meta_graph_coloring_antcol/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All libraries imported successfully!


## 2. Study Configuration

Define dataset selection, study name, and hyperparameter search space.

**Parameter Categories:**
- **Core ACO**: iterations, alpha, beta, rho, ant_count, Q, patience
- **Exploitation/Exploration**: exploitation_ratio (ant distribution balance)
- **Heuristic Weights**: color_preference_weight, constraint_penalty_weight
- **Pheromone Bounds**: min_pheromone, max_pheromone
- **Strategy Config**: num_strategies (number of node ordering strategies)
- **Warm-up Control**: use_warmup (enable/disable warm-up phase)

In [5]:
# Dataset selection: 'tiny_dataset' for quick testing, 'main_dataset' for full experiments
DATASET_NAME = 'tiny_dataset'  # Change to 'main_dataset' for full tuning

# Study continuation mode
# Option 1: Continue existing study - set CONTINUE_STUDY to the study name
# Example: CONTINUE_STUDY = 'aco_study_tiny_dataset_20251129_172927'
# Option 2: Start new study - set CONTINUE_STUDY = None
CONTINUE_STUDY = None  # Set to study name to continue, or None for new study

# Study name (used for new studies or when CONTINUE_STUDY is None)
if CONTINUE_STUDY:
    STUDY_NAME = CONTINUE_STUDY
    print(f"Continuing existing study: {STUDY_NAME}")
else:
    STUDY_NAME = f'aco_study_{DATASET_NAME}_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
    print(f"Creating new study: {STUDY_NAME}")

# Number of Optuna trials for hyperparameter tuning
N_TRIALS = 30

# Number of random exploration trials before optimization starts
N_STARTUP_TRIALS = 4

# ACO verbose setting 
ACO_VERBOSE = False   # Set to True to see detailed ACO progress

# Hyperparameter search space configuration (parameters to optimize)
PARAM_CONFIG = {
    # === Core ACO Parameters ===
    'iterations': {
        'type': 'int',
        'low': 100,
        'high': 300,
    },
    'alpha': {
        'type': 'float',
        'low': 0.5,
        'high': 3.0,
    },
    'beta': {
        'type': 'float',
        'low': 1.0,
        'high': 6.0,
    },
    'rho': {
        'type': 'float',
        'low': 0.01,
        'high': 0.4,
    },
    'ant_count': {
        'type': 'int',
        'low': 20,
        'high': 100,
    },
    'Q': {
        'type': 'float',
        'low': 1.0,
        'high': 10.0,
    },
    'patience': {
        'type': 'float',
        'low': 0.50,
        'high': 0.95,
    },
    
    # === Exploitation/Exploration Balance ===
    'exploitation_ratio': {
        'type': 'float',
        'low': 0.3,
        'high': 0.8,
        'description': 'Ratio of pheromone-following ants (exploitation) vs anti-pheromone ants (exploration)'
    },
    
    # === Heuristic Weight Parameters ===
    'color_preference_weight': {
        'type': 'float',
        'low': 5.0,
        'high': 25.0,
        'description': 'Weight for preferring existing colors over new ones'
    },
    'constraint_penalty_weight': {
        'type': 'float',
        'low': 0.05,
        'high': 0.5,
        'description': 'Penalty for colors that constrain neighbor choices'
    },
    
    # === Pheromone Bounds ===
    'min_pheromone': {
        'type': 'float',
        'low': 0.01,
        'high': 0.2,
        'description': 'Minimum pheromone level (prevents stagnation)'
    },
    'max_pheromone': {
        'type': 'float',
        'low': 5.0,
        'high': 15.0,
        'description': 'Maximum pheromone level (prevents dominance)'
    },
    
    # === Strategy Parameters ===
    'num_strategies': {
        'type': 'categorical',
        'choices': [2, 3, 4],
        'description': 'Number of node ordering strategies for greedy ants'
    },
    
    # === Warm-up Control ===
    'use_warmup': {
        'type': 'categorical',
        'choices': [True, False],
        'description': 'Enable/disable warm-up phase (greedy exploration before ACO)'
    }
}

print(f"Configuration:")
print(f"  Dataset: {DATASET_NAME}")
print(f"  Study Name: {STUDY_NAME}")
print(f"  Optuna Trials: {N_TRIALS}")
print(f"  Random Exploration Trials: {N_STARTUP_TRIALS} (before optimization)")
print(f"\nHyperparameters to Optimize:")
print(f"  Core ACO Parameters:")
for param_name in ['iterations', 'alpha', 'beta', 'rho', 'ant_count', 'Q', 'patience']:
    param_spec = PARAM_CONFIG[param_name]
    print(f"    {param_name}: [{param_spec['low']}, {param_spec['high']}] ({param_spec['type']})")

print(f"\n  Exploitation/Exploration Balance:")
param_spec = PARAM_CONFIG['exploitation_ratio']
print(f"    exploitation_ratio: [{param_spec['low']}, {param_spec['high']}] - {param_spec.get('description', '')}")

print(f"\n  Heuristic Weights:")
for param_name in ['color_preference_weight', 'constraint_penalty_weight']:
    param_spec = PARAM_CONFIG[param_name]
    print(f"    {param_name}: [{param_spec['low']}, {param_spec['high']}] - {param_spec.get('description', '')}")

print(f"\n  Pheromone Bounds:")
for param_name in ['min_pheromone', 'max_pheromone']:
    param_spec = PARAM_CONFIG[param_name]
    print(f"    {param_name}: [{param_spec['low']}, {param_spec['high']}] - {param_spec.get('description', '')}")

print(f"\n  Strategy Parameters:")
param_spec = PARAM_CONFIG['num_strategies']
print(f"    num_strategies: {param_spec['choices']} - {param_spec.get('description', '')}")

print(f"\n  Warm-up Control:")
param_spec = PARAM_CONFIG['use_warmup']
print(f"    use_warmup: {param_spec['choices']} - {param_spec.get('description', '')}")

Creating new study: aco_study_tiny_dataset_20251204_161931
Configuration:
  Dataset: tiny_dataset
  Study Name: aco_study_tiny_dataset_20251204_161931
  Optuna Trials: 30
  Random Exploration Trials: 4 (before optimization)

Hyperparameters to Optimize:
  Core ACO Parameters:
    iterations: [100, 300] (int)
    alpha: [0.5, 3.0] (float)
    beta: [1.0, 6.0] (float)
    rho: [0.01, 0.4] (float)
    ant_count: [20, 100] (int)
    Q: [1.0, 10.0] (float)
    patience: [0.5, 0.95] (float)

  Exploitation/Exploration Balance:
    exploitation_ratio: [0.3, 0.8] - Ratio of pheromone-following ants (exploitation) vs anti-pheromone ants (exploration)

  Heuristic Weights:
    color_preference_weight: [5.0, 25.0] - Weight for preferring existing colors over new ones
    constraint_penalty_weight: [0.05, 0.5] - Penalty for colors that constrain neighbor choices

  Pheromone Bounds:
    min_pheromone: [0.01, 0.2] - Minimum pheromone level (prevents stagnation)
    max_pheromone: [5.0, 15.0] - Maxi

### ACO Algorithm Architecture

**Two-Phase Strategy:**

1. **Warm-Up Phase** (Greedy Exploration) - *Optional via use_warmup parameter*
   - Duration: Auto-calculated based on graph size
   - Coverage: N × (1 + num_strategies) scenarios
     - S-START: Each node as starting point (N scenarios)
     - STRATEGIES: Each node with each ordering strategy (N × num_strategies scenarios)
   - Ant deployment: Only greedy ants (systematic coverage)
   - Pheromone: NO updates during warm-up
   - Purpose: Exhaust ALL greedy possibilities before ACO begins

2. **Post-Warmup Phase** (ACO Refinement)
   - Duration: Remaining iterations after warm-up (or all iterations if use_warmup=False)
   - Ant deployment: 2 groups controlled by exploitation_ratio parameter
     - Pheromone ants: exploitation_ratio × ant_count (exploit learned knowledge)
     - Anti-pheromone ants: (1 - exploitation_ratio) × ant_count (explore alternatives)
   - Pheromone: Active updates (evaporation + selective reinforcement)
   - Purpose: Pure ACO optimization with configurable exploitation/exploration balance

**Example**: 100 nodes, 30 ants, 3 strategies, exploitation_ratio=0.6, use_warmup=True
- Warm-up: 40 iterations (400 scenarios ÷ 10 greedy ants)
- Post-warmup: Remaining iterations with 18 pheromone + 12 anti-pheromone ants (60%/40% split)

In [6]:
# Initialize the Optuna tuner
tuner = OptunaACOTuner(
    study_name=STUDY_NAME,
    data_root=str(DATA_ROOT),
    direction='minimize',  # We want to minimize the number of colors used
    n_startup_trials=N_STARTUP_TRIALS  # Number of random trials before optimization
)

print(f"Tuner initialized with study: {STUDY_NAME}")
print(f"Data root: {DATA_ROOT}")

# Load tuning dataset once (before optimization)
print("\nLoading tuning dataset...")
data_loader = GraphDataLoader(str(DATA_ROOT), DATASET_NAME)
tuning_graphs, tabu_best_values = data_loader.load_tuning_dataset()
print(f"Loaded {len(tuning_graphs)} graphs for tuning\n")

# Wrapper function to pass tuning graphs and tabu best values to objective function
def objective_wrapper(trial, params, **kwargs):
    return aco_objective_function(
        trial=trial,
        params=params,
        tuning_graphs=tuning_graphs,
        aco_class=ACOGraphColoring,
        verbose=ACO_VERBOSE,
        tabu_best_values=tabu_best_values,
        n_startup_trials=N_STARTUP_TRIALS
    )

print("Objective function wrapper ready!")

Tuner initialized with study: aco_study_tiny_dataset_20251204_161931
Data root: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data

Loading tuning dataset...

Loading Tuning Dataset: tiny_dataset
  gc_20_9:
    Nodes: 20, Edges: 164
    Density: 0.8632, Connected: True
  gc_50_9:
    Nodes: 50, Edges: 1103
    Density: 0.9004, Connected: True

Graphs ordered by size (smallest to largest):
  1. gc_20_9: 20 nodes, 164 edges
  2. gc_50_9: 50 nodes, 1103 edges


Loaded Tabu Tuning Best Results: 8 graphs
  gc_1000_7: 195 colors (from tabu search tuning)
  gc_100_9: 38 colors (from tabu search tuning)
  gc_20_9: 10 colors (from tabu search tuning)
  gc_250_9: 85 colors (from tabu search tuning)
  gc_4_1: 2 colors (from tabu search tuning)
  gc_500_9: 162 colors (from tabu search tuning)
  gc_50_9: 21 colors (from tabu search tuning)
  gc_70_9: 28 colors (from tabu search tuning)

Loaded 2 graphs for tuning

Objective function wrapper ready!


## 3. Run Hyperparmeters Tuning

In [7]:
# Run the hyperparameter optimization
print(f"\nStarting hyperparameter optimization with {N_TRIALS} trials...")
print("=" * 70)

best_params = tuner.optimize(
    objective_func=objective_wrapper,
    param_config=PARAM_CONFIG,
    aco_class=ACOGraphColoring,
    n_trials=N_TRIALS,
    n_jobs=6
)

print("\n" + "=" * 70)
print("Optimization completed!")
print("\nBest Parameters Found:")
for param_name, param_value in best_params.items():
    print(f"  {param_name}: {param_value}")

[I 2025-12-04 16:19:31,961] A new study created in Journal with name: aco_study_tiny_dataset_20251204_161931



Starting hyperparameter optimization with 30 trials...
✓ Created new study 'aco_study_tiny_dataset_20251204_161931'
  Storage: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_tiny_dataset_20251204_161931/aco_study_tiny_dataset_20251204_161931.log
  Random exploration trials: 4 (before optimization)
Trial Summary:
  Successful: 0/30
  Total executed: 0
  Remaining needed: 30

Starting optimization with 30 trials...


  0%|          | 0/30 [00:00<?, ?it/s]


Trial 2: Random Exploration (Startup 3/4)
Parameters:
  iterations: 261
  alpha: 1.4245430511636217
  beta: 5.067762959432656
  rho: 0.06946314221504594
  ant_count: 67
  Q: 6.814376167542012
  patience: 0.94731100828498
  exploitation_ratio: 0.6044035635825149
  color_preference_weight: 15.21011745592904
  constraint_penalty_weight: 0.28570399378973066
  min_pheromone: 0.11056170239747777
  max_pheromone: 14.10032781856334
  num_strategies: 2
  use_warmup: False
----------------------------------------------------------------------

💡 Trial 2 | Graph 'gc_20_9' | [1/2] | Starting (nodes=20, edges=164, tabu_best=10)

Trial 1: Random Exploration (Startup 2/4)
Parameters:
  iterations: 232
  alpha: 2.3631977550434007
  beta: 3.134345996295644
  rho: 0.2982347421666359
  ant_count: 65
  Q: 1.1492035732616577
  patience: 0.8495735013134191
  exploitation_ratio: 0.5016289206636492
  color_preference_weight: 10.923337123293724
  constraint_penalty_weight: 0.11590950213542647
  min_pheromone:

  0%|          | 0/30 [03:51<?, ?it/s]

...

Best trial: 0. Best value: 33:  97%|█████████▋| 29/30 [45:13<01:15, 75.97s/it]

🔹 Trial 29 | Graph 'gc_50_9' | [2/2] | ✓ Result: 22 colors in 249 iterations - ⚠️ Less than Tabu Best: 21 (+1) | ⏱ 251.14s

----------------------------------------------------------------------
Trial 29 Summary:
  Total colors across all graphs: 33
  Average colors per graph: 16.50

[I 2025-12-04 17:04:45,917] Trial 29 finished with value: 33.0 and parameters: {'iterations': 249, 'alpha': 1.421796304430591, 'beta': 1.773316815566527, 'rho': 0.11754786177082778, 'ant_count': 66, 'Q': 6.49200847925762, 'patience': 0.9442699786094246, 'exploitation_ratio': 0.5982523388415618, 'color_preference_weight': 17.89449397377743, 'constraint_penalty_weight': 0.3022727864603943, 'min_pheromone': 0.11244744247749178, 'max_pheromone': 11.794529313677394, 'num_strategies': 2, 'use_warmup': False}. Best is trial 0 with value: 33.0.


Best trial: 0. Best value: 33: 100%|██████████| 30/30 [45:14<00:00, 90.48s/it]

    Saved trial visualizations: 3 metric plots, 2 graph plots

Optimization complete!
Best value: 33.0
Best parameters: {'iterations': 114, 'alpha': 2.643758337847659, 'beta': 1.0511555179963052, 'rho': 0.15874788910789756, 'ant_count': 93, 'Q': 3.1236696831649824, 'patience': 0.844118738585755, 'exploitation_ratio': 0.7686251569326721, 'color_preference_weight': 16.180318913181495, 'constraint_penalty_weight': 0.1881625468526913, 'min_pheromone': 0.01682453022087227, 'max_pheromone': 10.81707322465002, 'num_strategies': 3, 'use_warmup': True}
Study summary saved to: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_tiny_dataset_20251204_161931/aco_study_tiny_dataset_20251204_161931_summary.json

Generating visualization plots...


Optimization history saved to: history.png
Could not plot parameter importances: Encountered zero total variance in all trees.
Slice plot saved to: slice.png
Timeline plot saved to: timeline.png
All plots generated successfully.


Optimization completed!

Best Parameters Found:
  iterations: 114
  alpha: 2.643758337847659
  beta: 1.0511555179963052
  rho: 0.15874788910789756
  ant_count: 93
  Q: 3.1236696831649824
  patience: 0.844118738585755
  exploitation_ratio: 0.7686251569326721
  color_preference_weight: 16.180318913181495
  constraint_penalty_weight: 0.1881625468526913
  min_pheromone: 0.01682453022087227
  max_pheromone: 10.81707322465002
  num_strategies: 3
  use_warmup: True


In [8]:
# Generate all optimization plots
print("\nGenerating optimization visualization plots...")
print("=" * 70)
tuner.generate_plots(recreate=True)
print("=" * 70)


Generating optimization visualization plots...

Generating visualization plots...
Optimization history saved to: history.png
Could not plot parameter importances: Encountered zero total variance in all trees.
Slice plot saved to: slice.png
Timeline plot saved to: timeline.png
All plots generated successfully.



### Best Trial Results

In [9]:
# Display best trial graphs and metrics


# Get best trial number
best_trial_number = tuner.study.best_trial.number
best_trial_dir = Path(DATA_ROOT) / 'studies' / STUDY_NAME / 'results' / f'trial_{best_trial_number:04d}'

print(f"Best Trial: {best_trial_number}")
print(f"Best Objective Value: {tuner.study.best_value}")
print("=" * 70)

# Display metrics for best trial
display(Markdown(f"#### Best Trial Metrics"))

metric_files = [
    ('color_count.png', 'Color Count per Graph'),
    ('execution_time.png', 'Execution Time per Graph'),
    ('conflicts.png', 'Conflicts per Graph')
]

for filename, title in metric_files:
    metric_path = best_trial_dir / filename
    if metric_path.exists():
        display(Markdown(f"**{title}**"))
        display(HTML(f'<img src="{metric_path}" style="width: 75%; display: block; margin: 0 auto;">'))

# Display colored graphs for best trial
display(Markdown(f"#### Best Trial: Colored Graph Solutions"))

# Find all graph files in best trial directory
graph_files = sorted(best_trial_dir.glob('graph_*.png'))
for graph_file in graph_files:
    graph_name = graph_file.stem.replace('graph_', '')
    display(Markdown(f"**{graph_name}**"))
    display(HTML(f'<img src="{graph_file}" style="width: 75%; display: block; margin: 0 auto;">'))

Best Trial: 0
Best Objective Value: 33.0


#### Best Trial Metrics

**Color Count per Graph**

**Execution Time per Graph**

**Conflicts per Graph**

#### Best Trial: Colored Graph Solutions

**gc_20_9**

**gc_50_9**

### Display Study Visualizations

In [10]:
# Display all study figures

study_figures_path = Path(DATA_ROOT) / 'studies' / STUDY_NAME / 'figures'

# List of all possible study figures with descriptions
figure_files = [
    ('history.png', 'Optimization History', 'Shows objective value progression across trials'),
    ('importances.png', 'Parameter Importances', 'Shows which hyperparameters impact results most'),
    ('slice.png', 'Slice Plots (All Parameters)', 'Shows how each parameter affects objective value'),
    ('timeline.png', 'Trial Timeline', 'Shows when each trial ran and how long it took')
]

print("Study Visualization Figures:")
print("=" * 70)

for filename, title, description in figure_files:
    figure_path = study_figures_path / filename
    if figure_path.exists():
        display(Markdown(f"### {title}"))
        display(Markdown(f"*{description}*"))
        display(HTML(f'<img src="{figure_path}" style="width: 75%; display: block; margin: 0 auto;">'))
        print(f"✓ {filename}")
    else:
        print(f"✗ {filename} (not generated - may require additional dependencies)")

print("=" * 70)

Study Visualization Figures:


### Optimization History

*Shows objective value progression across trials*

✓ history.png
✗ importances.png (not generated - may require additional dependencies)


### Slice Plots (All Parameters)

*Shows how each parameter affects objective value*

✓ slice.png


### Trial Timeline

*Shows when each trial ran and how long it took*

✓ timeline.png


## 4. Comprehensive Algorithms Comparison

Compare three algorithms on the testing dataset:
- **Greedy Algorithm**: Fast constructive heuristic
- **Tabu Search**: Single-solution metaheuristic with memory
- **ACO**: Population-based metaheuristic (using best parameters from tuning)

Each algorithm runs 5 times on all test graphs to calculate statistics and generate comparison plots.

In [11]:
# Import the comprehensive testing module
from comprehensive_testing import run_comprehensive_testing

# Set up testing directories - save comparison inside study path
comparison_output_dir = DATA_ROOT / 'studies' / STUDY_NAME / 'algorithm_comparison'

print(f"Data root: {DATA_ROOT}")
print(f"Dataset: {DATASET_NAME}")
print(f"Comparison output directory: {comparison_output_dir}")
print(f"  (Results will be saved in study: {STUDY_NAME})")

Data root: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data
Dataset: tiny_dataset
Comparison output directory: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_tiny_dataset_20251204_161931/algorithm_comparison
  (Results will be saved in study: aco_study_tiny_dataset_20251204_161931)


In [12]:
# Prepare ACO parameters from best trial
aco_test_params = {
    'iterations': int(best_params['iterations'])*5,
    'alpha': best_params['alpha'],
    'beta': best_params['beta'],
    'rho': best_params['rho'],
    'ant_count': int(best_params['ant_count']),
    'Q': best_params['Q'],
    'patience': best_params['patience']
}

print("\nACO Parameters for Testing (from best trial):")
for param, value in aco_test_params.items():
    print(f"  {param}: {value}")


ACO Parameters for Testing (from best trial):
  iterations: 570
  alpha: 2.643758337847659
  beta: 1.0511555179963052
  rho: 0.15874788910789756
  ant_count: 93
  Q: 3.1236696831649824
  patience: 0.844118738585755


In [13]:
# Run comprehensive testing: 5 repetitions per algorithm per graph
# Results are cached in JSON - set force_rerun=True to re-execute tests
NUM_REPETITIONS = 1
FORCE_RERUN = False  # Set to True to ignore cached results and rerun testing

print(f"\n{'='*80}")
print(f"Running comprehensive testing with {NUM_REPETITIONS} repetitions per algorithm")
if FORCE_RERUN:
    print("⚠ FORCE_RERUN=True: Will ignore cached results and rerun all tests")
else:
    print("ℹ Using cached results if available (set FORCE_RERUN=True to rerun)")
print(f"{'='*80}\n")

df_comparison_results, df_comparison_stats = run_comprehensive_testing(
    data_root=str(DATA_ROOT),
    dataset_name=DATASET_NAME,
    output_dir=str(comparison_output_dir),
    num_repetitions=NUM_REPETITIONS,
    aco_params=aco_test_params,
    force_rerun=FORCE_RERUN
)

print("\n✓ Comprehensive testing complete!")


Running comprehensive testing with 1 repetitions per algorithm
ℹ Using cached results if available (set FORCE_RERUN=True to rerun)


Loaded Best Known Results: 8 graphs
  gc_1000_7: 195 colors
  gc_100_9: 38 colors
  gc_20_9: 10 colors
  gc_250_9: 85 colors
  gc_4_1: 2 colors
  gc_500_9: 162 colors
  gc_50_9: 21 colors
  gc_70_9: 28 colors


Loading Testing Dataset: tiny_dataset
  gc_70_9:
    Nodes: 70, Edges: 2158
    Density: 0.8936, Connected: True

Graphs ordered by size (smallest to largest):
  1. gc_70_9: 70 nodes, 2158 edges

Checking for existing test results...
COMPREHENSIVE ALGORITHM TESTING
Found 1 test graphs
Running 1 repetitions per graph per algorithm

Processing: gc_70_9
  Nodes: 70, Edges: 2158
  Running Greedy...
    Rep 1: colors=32, conflicts=0, time=0.002s
  Running Tabu Search...
    Rep 1: colors=30, conflicts=0, time=25.686s
  Running ACO...
💬 Iteration 100/570: iter_best=33, global_best=29
💬 Iteration 200/570: iter_best=32, global_best=29

❌  Early stopping: 

### 6.1 Display Statistics Summary

In [14]:
# Display statistics table
from IPython.display import display, Markdown

print("Statistics Summary:")
display(df_comparison_stats)

# Print formatted statistics (without markdown)
print("\n" + "="*80)
print(df_comparison_stats.to_string(index=False))
print("="*80)

Statistics Summary:


,graph,best_known,algorithm,best_colors,avg_colors,std_colors,avg_conflicts,avg_time,std_time,deviation_best,deviation_avg
0,gc_70_9,28,Greedy,32,32.0,NaN,0.0,0.001792,NaN,14.285714,14.285714
1,gc_70_9,28,Tabu Search,30,30.0,NaN,0.0,25.685939,NaN,7.142857,7.142857
2,gc_70_9,28,ACO,29,29.0,NaN,0.0,318.576238,NaN,3.571429,3.571429



  graph  best_known   algorithm  best_colors  avg_colors  std_colors  avg_conflicts   avg_time  std_time  deviation_best  deviation_avg
gc_70_9          28      Greedy           32        32.0         NaN            0.0   0.001792       NaN       14.285714      14.285714
gc_70_9          28 Tabu Search           30        30.0         NaN            0.0  25.685939       NaN        7.142857       7.142857
gc_70_9          28         ACO           29        29.0         NaN            0.0 318.576238       NaN        3.571429       3.571429


### 6.2 Display Comparison Visualizations

In [15]:
# Display algorithm comparison plots (4 comparison metrics)
from IPython.display import Image

# Define all comparison plots
comparison_plots = [
    ('comparison_best_colors.png', 'Best Color Count Comparison (with Best Known Solutions)'),
    ('comparison_avg_colors.png', 'Average Color Count Comparison (with Std Dev)'),
    ('comparison_execution_time.png', 'Execution Time Comparison (log scale)'),
    ('comparison_conflicts.png', 'Conflict Count Comparison')
]

print("Algorithm Comparison Metrics:")
print("="*80)

for filename, title in comparison_plots:
    plot_path = comparison_output_dir / filename
    if plot_path.exists():
        display(Markdown(f"### {title}"))
        display(Markdown(f'<img src="{plot_path}" style="width: 75%; display: block; margin: 0 auto;"/>'))
    else:
        print(f"⚠ {title} not found: {filename}")

print("="*80)

Algorithm Comparison Metrics:


### Best Color Count Comparison (with Best Known Solutions)

<img src="/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_tiny_dataset_20251204_161931/algorithm_comparison/comparison_best_colors.png" style="width: 75%; display: block; margin: 0 auto;"/>

### Average Color Count Comparison (with Std Dev)

<img src="/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_tiny_dataset_20251204_161931/algorithm_comparison/comparison_avg_colors.png" style="width: 75%; display: block; margin: 0 auto;"/>

### Execution Time Comparison (log scale)

<img src="/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_tiny_dataset_20251204_161931/algorithm_comparison/comparison_execution_time.png" style="width: 75%; display: block; margin: 0 auto;"/>

### Conflict Count Comparison

<img src="/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_tiny_dataset_20251204_161931/algorithm_comparison/comparison_conflicts.png" style="width: 75%; display: block; margin: 0 auto;"/>

### 6.3 Results Summary and File Locations

In [16]:
# Print comprehensive testing results summary
print("="*80)
print("COMPREHENSIVE TESTING RESULTS SUMMARY")
print("="*80)

print(f"\n📁 Output Directory: {comparison_output_dir}")
print(f"   (Saved in study: {STUDY_NAME})")

print(f"\n📊 Generated Files:")
print(f"   - comparison_results.json (complete results - cached for reuse)")
print(f"   - comprehensive_test_results.csv (all {len(df_comparison_results)} individual runs)")
print(f"   - statistics_summary.csv (aggregated statistics)")
print(f"   - comparison_best_colors.png (best color count comparison)")
print(f"   - comparison_avg_colors.png (average color count with std dev)")
print(f"   - comparison_execution_time.png (execution time comparison)")
print(f"   - comparison_conflicts.png (conflict count comparison)")
print(f"   - color_distribution_boxplots.png (distribution analysis)")

print(f"\n💡 Tip: Results are cached in JSON. Re-running this cell will use cached data.")
print(f"   Set FORCE_RERUN=True to ignore cache and rerun all tests.")

print(f"\n🎯 Summary by Algorithm:")
for algo_name in df_comparison_stats['algorithm'].unique():
    df_algo = df_comparison_stats[df_comparison_stats['algorithm'] == algo_name]
    best_colors = df_algo['best_colors'].min()
    avg_colors = df_algo['avg_colors'].mean()
    avg_time = df_algo['avg_time'].mean()
    print(f"   {algo_name}:")
    print(f"      Best colors: {best_colors}")
    print(f"      Avg colors: {avg_colors:.2f}")
    print(f"      Avg time: {avg_time:.4f}s")

print("\n" + "="*80)

COMPREHENSIVE TESTING RESULTS SUMMARY

📁 Output Directory: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_tiny_dataset_20251204_161931/algorithm_comparison
   (Saved in study: aco_study_tiny_dataset_20251204_161931)

📊 Generated Files:
   - comparison_results.json (complete results - cached for reuse)
   - comprehensive_test_results.csv (all 3 individual runs)
   - statistics_summary.csv (aggregated statistics)
   - comparison_best_colors.png (best color count comparison)
   - comparison_avg_colors.png (average color count with std dev)
   - comparison_execution_time.png (execution time comparison)
   - comparison_conflicts.png (conflict count comparison)
   - color_distribution_boxplots.png (distribution analysis)

💡 Tip: Results are cached in JSON. Re-running this cell will use cached data.
   Set FORCE_RERUN=True to ignore cache and rerun all tests.

🎯 Summary by Algorithm:
   Greedy:
      Best colors: 32
      Avg colors: 32.00
      Avg time: 